In [ ]:
import requests
import json
import pickle



def get_bne_wikidata(max_number=100):
    key = "bne"
    value = "wikidata"
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
    SELECT ?bne ?wikidata
    WHERE { 
        ?wikidata wdt:P950 ?bne .
    }
    LIMIT """+str(max_number)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    json_data = response.json()['results']['bindings']
    return [(item[key]['value'], item[value]['value']) for item in json_data]


datos = get_bne_wikidata()
#for key,value in datos:
    #print(key,"\t",value)

f = open("wikidatamap.bin", "wb+")
pickle.dump(datos, f)

In [3]:
import importlib
import pickle
import dataset
importlib.reload(dataset)


datos = dataset.Dataset()
query2 = """PREFIX wikibase: <http://wikiba.se/ontology>
construct { ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }
WHERE { ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
    ?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
} LIMIT 10000 OFFSET 1880000
"""
datos.load_dataset_from_query(query2, only_uri=False)


#datos.load_dataset_from_nlevels(5, extra_params="LIMIT 100")
datos.show(verbose=False)
dataset1 = datos.train_split()

#     print(sub,dataset1[sub])
datos.save_to_binary("wikidata_2.bin")

# datos.subs

#print(len(jsonlist))
#jsonlist

Exception: Error on endpoint. HTTP status code: 500

In [164]:
import numpy as np
rel = [i[2] for i in datos.subs]
print(len(datos.relations_dict))
print(len(datos.relations))
print(len(datos.entities_dict))
print(len(datos.entities))
rel = np.array(rel)
print(np.sort(rel))
data = np.matrix(datos.subs)
print(data[:,2])

print('Splitting the data into training and validation sets ...')
print(data.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
nb_validation_samples = int(0.2 * data.shape[0])


x_train = data[:-nb_validation_samples]
x_val = data[-nb_validation_samples:-int(nb_validation_samples/2)]
x_test = data[-int(nb_validation_samples/2):]

print(x_val.shape,"\n\n\n",x_test.shape,"\n\n\n",x_train.shape)
print(x_test.shape[0]+x_train.shape[0]+x_val.shape[0])


440
440
1497
1497
[  0   0   0 ..., 438 438 439]
[[  0]
 [  1]
 [  3]
 ..., 
 [220]
 [439]
 [ 31]]
Splitting the data into training and validation sets ...
(1727, 3)
(173, 3) 


 (172, 3) 


 (1382, 3)
1727


In [ ]:
import importlib
import pickle
import dataset
importlib.reload(dataset)


datos = dataset.Dataset()
datos.load_from_binary("wdata2lvl.bin")
datos.show(verbose=False)

In [17]:
import requests

def get_json_offset(n, count):
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
construct {{ ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }}
WHERE {{ ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
}} LIMIT {0} OFFSET {1}
""".format(count, n*count)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    if response.status_code is not 200:
        return False, "Error occurred on http request. Code"+str(response.status_code)
    json_data = response.json()['results']['bindings']
    return json_data

all_json = []
for query in range(0,30):
    json1 = get_json_offset(query,20000)
    all_json = all_json + json1
    


In [6]:
import importlib
import dataset
import queries
import pickle
importlib.reload(dataset)
importlib.reload(queries)

query = queries.Queries()

js = query.big_query(n_levels=3, n_rounds=4, total=10000)
print("finished. Saving")
f1 = open("millon.json", "wb+")
pickle.dump(js, f1)
f1.close()

R0: Offset: 0, limit: 2500
R1: Offset: 2500, limit: 2500
R2: Offset: 5000, limit: 2500
R3: Offset: 7500, limit: 2500
finished. Saving


In [4]:
import importlib
import dataset
import pickle
importlib.reload(dataset)
from datetime import datetime

dtset = dataset.Dataset()


#dataset.load_entire_dataset(1)
dtset.load_dataset_recurrently(3, verbose=2)
dtset.show()



Found 32563 entities
Scanning level 1 with 32701 elements
Waiting all threads to end
Scanning level 2 with 11560 elements
Waiting all threads to end
Scanning level 3 with 1577 elements
Waiting all threads to end
13767 entities, 730 relations, 15079 tripletas
Enter S to show status: q


In [5]:
dtset.show()
dtset.save_to_binary("wdata_15k.bin")

13767 entities, 730 relations, 15079 tripletas


True

In [2]:
import importlib
import dataset
import pickle
importlib.reload(dataset)
from datetime import datetime

dataset = dataset.Dataset()


#dataset.load_entire_dataset(1)
dataset.load_from_binary("wikidata_1.bin")
dataset.show()
dataset.save_to_binary("wikidata_10k.bin")

8363 entities, 930 relations, 9996 tripletas


True

In [1]:
times= [10, 60, 120, 180, 300, 420, 500, 600, 700, 800, 900]
try_catch_all = [548, 2635, None, 7216, 12142, 15622, None, 20500, 23986, 28086, 31591]
if_else_all=[550, 3369, 6771, 10057, 16666, 23575, 28325, None, None, None]
if_else_all2 = [818,3654,6924,17093,None,28298]

In [6]:
scores = [{'epoch': 10, 'score': 0.00092663197655540059, 'type': 'FMRR'}, {'epoch': 20, 'score': 0.0012139665638769258, 'type': 'FMRR'}, {'epoch': 30, 'score': 0.0012300529957259387, 'type': 'FMRR'}, {'epoch': 40, 'score': 0.0014716405210133468, 'type': 'FMRR'}, {'epoch': 50, 'score': 0.0011434153470357338, 'type': 'FMRR'}, {'epoch': 60, 'score': 0.000901344774094369, 'type': 'FMRR'}, {'epoch': 70, 'score': 0.00083572098045467987, 'type': 'FMRR'}, {'epoch': 80, 'score': 0.00076493759499324644, 'type': 'FMRR'}, {'epoch': 90, 'score': 0.00069111479648363897, 'type': 'FMRR'}, {'epoch': 100, 'score': 0.0007607878178162447, 'type': 'FMRR'}, {'epoch': 110, 'score': 0.00083800339831679158, 'type': 'FMRR'}, {'epoch': 120, 'score': 0.00093539114866330437, 'type': 'FMRR'}, {'epoch': 130, 'score': 0.00082375717574113881, 'type': 'FMRR'}, {'epoch': 140, 'score': 0.00095509207876159094, 'type': 'FMRR'}, {'epoch': 150, 'score': 0.0011681899014313088, 'type': 'FMRR'}, {'epoch': 160, 'score': 0.00097200143950825478, 'type': 'FMRR'}, {'epoch': 170, 'score': 0.0012535442024012538, 'type': 'FMRR'}, {'epoch': 180, 'score': 0.0011724069840263623, 'type': 'FMRR'}, {'epoch': 190, 'score': 0.00089371000741264302, 'type': 'FMRR'}, {'epoch': 200, 'score': 0.0011307594521970806, 'type': 'FMRR'}, {'epoch': 200, 'score': 0.0011307594521970806, 'type': 'FMRR'}]
tuples = [(e['score'], e['epoch']) for e in scores]
for t in sorted(tuples, key=lambda t: t[0], reverse=True) :
    print(t)

(0.0014716405210133468, 40)
(0.0012535442024012538, 170)
(0.0012300529957259387, 30)
(0.0012139665638769258, 20)
(0.0011724069840263623, 180)
(0.0011681899014313088, 150)
(0.0011434153470357338, 50)
(0.0011307594521970806, 200)
(0.0011307594521970806, 200)
(0.0009720014395082548, 160)
(0.0009550920787615909, 140)
(0.0009353911486633044, 120)
(0.0009266319765554006, 10)
(0.000901344774094369, 60)
(0.000893710007412643, 190)
(0.0008380033983167916, 110)
(0.0008357209804546799, 70)
(0.0008237571757411388, 130)
(0.0007649375949932464, 80)
(0.0007607878178162447, 100)
(0.000691114796483639, 90)


In [29]:
import itertools
times= [0.2, 2], [50,60,70,80,90,100,110,120,130,140,150],[True,False]
todo = list(itertools.product(*times))
print (len(todo))


44
